In [14]:
import numpy as np
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import optuna
%matplotlib inline

In [42]:
def load_data():
    data = pd.read_pickle('../../data/dataframes/data.pkl')
    nomination_onehot = pd.read_pickle('../../data/dataframes/nomination_onehot.pkl')
    selected_performers_onehot = pd.read_pickle('../../data/dataframes/selected_performers_onehot.pkl')
    selected_directors_onehot = pd.read_pickle('../../data/dataframes/selected_directors_onehot.pkl')
    selected_studio_onehot = pd.read_pickle('../../data/dataframes/selected_studio_onehot.pkl')
    selected_scriptwriter_onehot = pd.read_pickle('../../data/dataframes/selected_scriptwriter_onehot.pkl')
    review_dataframe = pd.read_pickle('../../data/dataframes/review_dataframe.pkl')

    
    # selected_directors_onehotとselected_scriptwriter_onehotの重複した人
    duplicate_scriptwriter = set(selected_directors_onehot.columns) & set(selected_scriptwriter_onehot.columns)
    selected_scriptwriter_onehot = selected_scriptwriter_onehot.drop(duplicate_scriptwriter, axis=1)
    
    frames = [nomination_onehot,selected_performers_onehot,selected_directors_onehot,selected_studio_onehot,selected_scriptwriter_onehot]

    df = data
    for f in frames:
        df = pd.merge(df, f, on='id')
    
    drop_elements = ["director", "other_nominates", "performers", "production_studio", "scriptwriter", "title",  'selected_performers', 'selected_directors', 'selected_studio',
       'selected_scriptwriter']
    df_drop = df.drop(drop_elements, axis=1)

    return df_drop, data

In [43]:
def standard(x_train, x_test):
    stdsc = StandardScaler()

    # 訓練用のデータを標準化
    x_train_std = stdsc.fit_transform(x_train)
    # 訓練用データを基準にテストデータも標準化
    x_test_std = stdsc.transform(x_test)
    
    return x_train_std, x_test_std

In [44]:
# 多重共線性の排除 (共分散0.7以上のfeatureを削除)

def collinearity(X):

    # 改善前
#     cor=np.corrcoef(X.T)
#     type(cor)
#     plt.figure(figsize=(20, 20))
#     sns.heatmap(cor, vmin=0.70,vmax=1,cmap=plt.cm.Spectral_r)

    # 改善
    drop_clm = ['吉田一夫']
    X = X.drop(drop_clm,  axis=1)

    # 改善後
    
#     cor=np.corrcoef(X.T)
#     type(cor)
#     plt.figure(figsize=(20, 20))
#     sns.heatmap(cor, vmin=0.80,vmax=1,cmap=plt.cm.Spectral_r)
    
    return X

In [45]:
def data_processing(test_year):
    df, data = load_data()
    
    df = collinearity(df)
    
    train = df.loc[df["year"] != test_year]
    test = df.loc[df["year"] == test_year]
    
    # 入力データの作成
    y_train = train['prize'].ravel()
    x_train = train.drop(['prize', 'year'], axis=1)
    
    features = x_train.columns
    
    x_train = x_train.values # 学習データ
    
    # テストデータの作成
    y_test = test['prize'].ravel()
    x_test = test.drop(['prize', 'year'], axis=1)
    x_test = x_test.values # テストデータ

    # データの標準化
    x_train_std, x_test_std = standard(x_train, x_test)
    
    return x_train_std, x_test_std, y_train, y_test, features

In [46]:
def print_weight_map(weight, features):
    # feature importance
    feature_imp = weight.reshape(features.shape)
    feature_imp_arg = np.argsort(feature_imp)
    f_num = len(feature_imp_arg)
    plt.rcParams["font.family"] = "IPAexGothic"
    plt.figure(figsize=(20,20))
    plt.barh(range(len(feature_imp_arg)), feature_imp[feature_imp_arg], color='b', align='center')
    plt.yticks(range(len(feature_imp_arg)), features[feature_imp_arg])
    plt.show()

In [47]:
def plot_roc_curve(fpr, tpr, auc):
    # ROC曲線をプロット
    plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
    plt.legend()
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.show()

In [48]:
# optunaで最適なC_valを探す

def train(trial):
    
    th_prob = 0.5
    C_val = trial.suggest_uniform('C_val', 0.0001, 0.001)
    
    print(" C_val:{}".format(C_val))
    
    cm_all = np.zeros((2, 2))
    
    # 予測した確率全体を格納
    probs_all = np.array([])
    y_true_all = np.array([])
    
    for y in range(1978, 2020):
        
        # データの生成
        x_train_std, x_test_std, y_train, y_test, features = data_processing(y)
        
        # 学習
        lr = LogisticRegression(class_weight="balanced", solver="liblinear",  penalty="l2", C=C_val) # ロジスティック回帰モデルのインスタンスを作成
        lr.fit(x_train_std, y_train) # ロジスティック回帰モデルの重みを学習
        
        # 予測(確率)
        probs = lr.predict_proba(x_test_std)
        probs_all = np.hstack((probs_all, probs[:,1]))
        y_true_all = np.hstack((y_true_all, y_test))

        # 混同行列
        y_pred = np.where((probs[:,1] ==  max(probs[:,1])) | (probs[:,1] > th_prob), 1, 0) #確率→0/1
        cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
        cm_all += cm
        
       # 各特徴量の重み係数
        weight = lr.coef_
        
        
#         print_weight_map(weight, features)

    auc = roc_auc_score(y_true_all, probs_all)
    fpr, tpr, thresholds = roc_curve(y_true_all, probs_all)
    
#     plot_roc_curve(fpr, tpr, auc)
    
    print("len: {0} , {1}".format(len(y_true_all), len(probs_all) ))
    print("confusion_matrix: ")
    print(cm_all)
    print("AUC: ")
    print(auc)
    print()
    return -auc

In [49]:
def main():
    np.set_printoptions(precision=3, suppress=True)
    study = optuna.create_study()
    study.optimize(train, n_trials=100)
    print(study.best_trial)

In [50]:
if __name__ == "__main__":
    main()

 C_val:0.00025271262055805117
len: 211 , 211
confusion_matrix: 
[[117.  52.]
 [ 11.  31.]]
AUC: 
0.7495069033530573



[I 2019-08-21 05:36:28,282] Finished trial#0 resulted in value: -0.7495069033530573. Current best value is -0.7495069033530573 with parameters: {'C_val': 0.00025271262055805117}.


 C_val:0.0006792193249104638
len: 211 , 211
confusion_matrix: 
[[116.  53.]
 [ 12.  30.]]
AUC: 
0.7455621301775147



[I 2019-08-21 05:36:36,496] Finished trial#1 resulted in value: -0.7455621301775147. Current best value is -0.7495069033530573 with parameters: {'C_val': 0.00025271262055805117}.


 C_val:0.0005948896837861526
len: 211 , 211
confusion_matrix: 
[[116.  53.]
 [ 12.  30.]]
AUC: 
0.7455621301775148



...

 C_val:0.00024499368217996767
len: 211 , 211
confusion_matrix: 
[[117.  52.]
 [ 11.  31.]]
AUC: 
0.7496477881093266



[I 2019-08-21 05:52:01,529] Finished trial#98 resulted in value: -0.7496477881093266. Current best value is -0.7503522118906735 with parameters: {'C_val': 0.00010009919430783164}.


 C_val:0.00014411440532777965
len: 211 , 211
confusion_matrix: 
[[117.  52.]
 [ 11.  31.]]
AUC: 
0.7500704423781347



[I 2019-08-21 05:52:13,576] Finished trial#99 resulted in value: -0.7500704423781347. Current best value is -0.7503522118906735 with parameters: {'C_val': 0.00010009919430783164}.


FrozenTrial(number=11, state=<TrialState.COMPLETE: 1>, value=-0.7503522118906735, datetime_start=datetime.datetime(2019, 8, 21, 5, 38, 4, 292138), datetime_complete=datetime.datetime(2019, 8, 21, 5, 38, 13, 996359), params={'C_val': 0.00010009919430783164}, distributions={'C_val': UniformDistribution(low=0.0001, high=0.001)}, user_attrs={}, system_attrs={'_number': 11}, intermediate_values={}, params_in_internal_repr={'C_val': 0.00010009919430783164}, trial_id=11)


In [ ]:
# l2 liblinear (共線性あり)
#  best value is -0.7550014088475626 with parameters: {'C_val': 0.00025433628983389037}.

# saga l1ration
# best value is -0.7434488588334741 with parameters: {'C_val': 0.00012028884882182497, 'l1_ratio': 0.0004895734071039444}.

#liblinear l2
# best value is -0.7503522118906735 with parameters: {'C_val': 0.00010009919430783164}.